## 1. Load dataset

In [1]:
import nltk
from nltk.tokenize import word_tokenize

In [2]:
from google.colab import drive
drive.mount('/content/drive') #Processed_Lemmatization_emotions.json

Mounted at /content/drive


In [304]:
import pandas as pd
#df = pd.read_json('/content/drive/MyDrive/DA-23/Pr_5/goodreads_reviews_spoiler.json', lines = True)
df = pd.read_json('/content/drive/MyDrive/DA-23/Pr_8/Processed_Lemmatization_emotions.json')

In [238]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19606 entries, 0 to 19998
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  19606 non-null  object
 1   Emotion   19606 non-null  object
dtypes: object(2)
memory usage: 459.5+ KB


In [239]:
df.describe()

,Sentence,Emotion
count,19606,19606
unique,19538,6
top,im feeling little stressed,joy
freq,3,6644


In [240]:
df['Emotion'].value_counts()

joy         6644
sadness     5675
anger       2640
fear        2318
love        1619
surprise     710
Name: Emotion, dtype: int64

## 2. Prepocessing

In [305]:
df_balanced = pd.DataFrame(columns=['Sentence','Emotion'])
df_balanced = df_balanced.append(df[df.Emotion=='joy'].sample(n=710, random_state=1, ignore_index=True)) # good
df_balanced = df_balanced.append(df[df.Emotion=='sadness'].sample(n=710, random_state=1, ignore_index=True)) # bad
df_balanced = df_balanced.append(df[df.Emotion=='anger'].sample(n=710, random_state=1, ignore_index=True)) # bad
df_balanced = df_balanced.append(df[df.Emotion=='fear'].sample(n=710, random_state=1, ignore_index=True)) # bad
df_balanced = df_balanced.append(df[df.Emotion=='love'].sample(n=710, random_state=1, ignore_index=True)) # good
df_balanced = df_balanced.append(df[df.Emotion=='surprise'].sample(n=710, random_state=1, ignore_index=True)) # good

<ipython-input-305-0bd07c102072>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_balanced = df_balanced.append(df[df.Emotion=='joy'].sample(n=710, random_state=1, ignore_index=True)) # good
<ipython-input-305-0bd07c102072>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_balanced = df_balanced.append(df[df.Emotion=='sadness'].sample(n=710, random_state=1, ignore_index=True)) # bad
<ipython-input-305-0bd07c102072>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_balanced = df_balanced.append(df[df.Emotion=='anger'].sample(n=710, random_state=1, ignore_index=True)) # bad
<ipython-input-305-0bd07c102072>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use p

In [306]:
df_balanced['Emotion'] = df_balanced['Emotion'].replace({'joy': '+','anger':'-', 'sadness': '-', 'fear': '-', 'love': '+', 'surprise': '+'})

In [326]:
df = df_balanced
df['Emotion'].value_counts()

+    2130
-    2130
Name: Emotion, dtype: int64

In [308]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [365]:
y = df['Emotion'].values
pd.get_dummies(df).astype('float32').values
enc = LabelEncoder().fit(y)
print(set(y))
y = enc.transform(y)
print("y:", set(y))
one_hot_labels = to_categorical(y)
print(one_hot_labels)

{'+', '-'}
y: {0, 1}
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [310]:
corpus = df['Sentence'].values
corpus

array(['feel really wierd suppose casual dating',
       'feel like continue bridge lesson since continuity going vital learn game',
       'feeling inspired write parody piece today much bad mood', ...,
       'bit lip slightly whispered feel weird tell hurt',
       'feel like weird sugar issue hunger place',
       'feeling overwhelmed trying think woman'], dtype=object)

In [327]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
ml = 10
nw = 1000

In [328]:
tokenizer = Tokenizer(num_words=nw, oov_token='<OOV>')
tokenizer.fit_on_texts(corpus)
word_index = tokenizer.word_index
sentences = tokenizer.texts_to_sequences(corpus)

In [329]:
sentences[:2]

[[2, 7, 1, 459, 1, 1], [2, 4, 670, 1, 671, 160, 1, 27, 922, 460, 563]]

In [330]:
padded = pad_sequences(sentences, maxlen=ml)
padded[:2]

array([[  0,   0,   0,   0,   2,   7,   1, 459,   1,   1],
       [  4, 670,   1, 671, 160,   1,  27, 922, 460, 563]], dtype=int32)

In [331]:
print(len(one_hot_labels))
print(len(padded))
print(type(one_hot_labels))
print(type(padded))

4260
4260
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


## 3. Model

In [351]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [386]:
#
def create_model(max_features = 10000,emb_dim = 128):
    model = keras.Sequential()
    model.add(layers.Embedding(max_features, emb_dim))
    model.add(layers.GRU(128))
    model.add(layers.Dense(2, activation='sigmoid'))
    model.compile(optimizer="adam",
                  loss=keras.losses.categorical_crossentropy,
                  metrics=["CategoricalAccuracy"])
    return model

def create_model_2(max_features = 10000,emb_dim = 128):
    model = keras.Sequential()
    model.add(layers.Embedding(max_features, emb_dim))
    model.add(layers.GRU(128))
    model.add(layers.Dense(2, activation='sigmoid'))
    model.compile(optimizer="adam",
                  loss=keras.losses.binary_crossentropy,
                  metrics=["accuracy"])
    return model


## 4. Prediction

In [387]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix

In [388]:
xtr, xts, ytr, yts = train_test_split(padded, to_categorical(y), test_size=0.2, random_state=42)

In [389]:
cv = CountVectorizer(max_features=nw)

In [390]:
print(xtr.shape)
print(ytr.shape)

print(xts.shape)
print(yts.shape)

(3408, 10)
(3408, 2)
(852, 10)
(852, 2)


In [391]:
model = create_model_2(max_features=nw)
print(model.summary())


Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_29 (Embedding)    (None, None, 128)         128000    
                                                                 
 gru (GRU)                   (None, 128)               99072     
                                                                 
 dense_29 (Dense)            (None, 2)                 258       
                                                                 
Total params: 227330 (888.01 KB)
Trainable params: 227330 (888.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [392]:
history_cv = model.fit(xtr,
                    ytr,
                    epochs=5,
                    batch_size=64,
                    validation_split=0.2,
                    shuffle = True,
                    verbose = 1)

Epoch 1/5
43/43 [==============================] - 8s 117ms/step - loss: 0.6805 - accuracy: 0.5682 - val_loss: 0.6209 - val_accuracy: 0.6686
Epoch 2/5
43/43 [==============================] - 1s 29ms/step - loss: 0.4164 - accuracy: 0.8239 - val_loss: 0.3944 - val_accuracy: 0.8270
Epoch 3/5
43/43 [==============================] - 1s 12ms/step - loss: 0.2139 - accuracy: 0.9149 - val_loss: 0.3745 - val_accuracy: 0.8490
Epoch 4/5
43/43 [==============================] - 1s 13ms/step - loss: 0.1538 - accuracy: 0.9336 - val_loss: 0.4837 - val_accuracy: 0.8504
Epoch 5/5
43/43 [==============================] - 1s 15ms/step - loss: 0.1216 - accuracy: 0.9494 - val_loss: 0.4672 - val_accuracy: 0.8519


In [393]:
predictions = model.predict(yts).argmax(axis=1)

27/27 [==============================] - 0s 2ms/step


In [394]:
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,